In [ ]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["ខ្ញុំស្រលាញ់កម្ពុជា", "ខ្ញុំចូលចិត្តកម្ពុជា"]

model = SentenceTransformer('sentence-transformers/LaBSE')
embeddings = model.encode(sentences)
print(embeddings)


In [ ]:
import pandas as pd 
import numpy as np 
df = pd.read_csv("AG_news_samples.csv")

In [ ]:
df_ = df[:50].copy()

In [ ]:
df_["embedded_title"] = df_["title"].apply(lambda x: model.encode(x))

In [ ]:
df_["embedded_desc"] = df_["description"].apply(lambda x: model.encode(x))

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
EMBED_MODEL = "all-MiniLM-L6-v2"
# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()


embedding_functions = embedding_functions.SentenceTransformerEmbeddingFunction(EMBED_MODEL)


In [ ]:
collection = client.get_or_create_collection("news-documents", embedding_function=embedding_functions,  metadata={"hnsw:space": "cosine"},)


In [ ]:
docuemnts = df_["description"].values.tolist()

In [ ]:
docuemnts

In [ ]:
collection.add(documents=docuemnts, ids=[f"id{i}" for i in range(len(df_))])

In [29]:
query_results = collection.query(
 query_texts=["pirate"],
   n_results=5,
 )
query_results

{'ids': [['id9', 'id17', 'id12', 'id10', 'id32']],
 'distances': [[0.6826481819152832,
   0.7472950220108032,
   0.7811960577964783,
   0.7954901456832886,
   0.8065580129623413]],
 'metadatas': [[{'genre': 'Sci/Tech'},
   {'genre': 'Sci/Tech'},
   {'genre': 'Sci/Tech'},
   {'genre': 'World'},
   {'genre': 'Sci/Tech'}]],
 'embeddings': None,
 'documents': [['The Microsoft CEO says one way to stem growing piracy of Windows and Office in emerging markets is to offer low-cost computers.',
   'A new, optional log-on service from America Online that makes it harder for scammers to access a persons online account will not be available for Macintosh ',
   "PalmSource surprised the mobile vendor community today with the announcement that it will acquire China MobileSoft (CMS), ostensibly to leverage that company's expertise in building a mobile version of the Linux operating system.",
   'RIYADH, Saudi Arabia -- Saudi police are seeking two young men in the killing of a Briton in a Riyadh park

In [ ]:
genres = [{"genre": g}for g in df_["label"]]

In [ ]:
collection.update(ids=[f"id{i}" for i in range(len(df_))], metadatas=genres)

In [ ]:
results = collection.query(query_texts=["Science"],  n_results=5)
results
